TODO's:
- Upload utility script somewhere, then have notebook fetch it (if it doesn't exist locally).
- Check Classifier performane on GPU (after tokenize moved to .foward). Does x need to be sent to device?
- Classifier save/load (inc. classifier trained on all data)


# ODSC 2022: An Introduction to Drift Detection

## Introduction




## 0. Getting ready

https://github.com/ascillitoe/odsc_workshop

```
git clone https://github.com/ascillitoe/odsc_workshop.git
```

In [102]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    import zipfile
    import shutil
    import os

    !wget -nc https://github.com/ascillitoe/odsc_workshop/archive/refs/heads/main.zip

    pz = open('main.zip', 'rb')
    packz = zipfile.ZipFile(pz)
    packz.extractall()
    pz.close()

    srcdir = 'odsc_workshop-main/'
    for filepath in os.listdir(srcdir):
        if not os.path.exists(filepath):
            shutil.copyfile(os.path.join(srcdir, filepath), filepath)
    shutil.rmtree(srcdir)

### Software

In this workshop we'll make use of state-of-the-art drift detectors from the open-source [Alibi Detect]() library. This can be installed (along with the PyTorch backend) via `pip`:

```
pip install alibi-detect[torch]
```

We'll also use a number of other packages which can also be installed with `pip`:

```
pip install umap-learn sentence-transformers statsmodels seaborn datasets scipy tqdm
```

You can also install the required dependencies by running the cell below:

In [ ]:
!pip install -r requirements.txt

### Download data and sentence transformer

In [ ]:
# Fetch preqrequisites
from workshop_utilities import fetch_prerequisites
dataset, sentence_transformer = fetch_prerequisites()

## 1. Classifying newsgroups

### The data

The 20 newsgroup dataset, which contains about 18,000 newsgroup posts across 20 topics, including politics, science sports and religion.

In [ ]:
print(f'{len(dataset.data)} documents')
print(f'{len(dataset.target_names)} categories:')
classes = dataset.target_names
classes

Let's take a look at an instance from the dataset:

In [ ]:
n = 1
for _, document in enumerate(dataset.data[:n]):
    category = dataset.target_names[dataset.target[_]]
    print(f'{_}. Category: {category}')
    print('---------------------------')
    print(document[:1000])
    print('---------------------------')

### Visualising the embeddings

We embed the news posts using [SentenceTransformers](https://www.sbert.net/index.html) pre-trained embeddings and optionally add a dimensionality reduction step with [UMAP](https://umap-learn.readthedocs.io/en/latest/).

In [ ]:
import numpy as np
from workshop_utilities import set_seed
set_seed(2022)  # This will ensure reproducibility (at least on CPU!)
n_all = len(dataset.data)

n_train = 5000  # can be reduced if too slow on cpu

idx_train = np.random.choice(n_all, size=n_train, replace=False)
x_train, y_train = [dataset.data[_] for _ in idx_train], dataset.target[idx_train]

In [ ]:
from workshop_utilities import EmbeddingModel
emb_model = EmbeddingModel(model=sentence_transformer)

In [ ]:
emb_train = emb_model(x_train)
emb_train.shape

By applying UMAP on the *SentenceTransformer* embeddings, we can visually inspect the various news topic clusters. UMAP is able to take advantage of our data labels (i.e. `y_train`).

In [ ]:
from workshop_utilities import UMAPModel, plot_clusters
umap_model = UMAPModel()
umap_model.fit(emb_train, y=y_train)
dr_train = umap_model.predict(emb_train)
dr_train.shape

In [ ]:
plot_clusters(dr_train, y_train, classes, title='Training data: clustered news topics')

### Training a classifier

First we train a classifier on a small subset of the data. The aim of the classifier is to predict the news topic of each instance.

Let's train our classifier. The classifier consists of a simple MLP head on top of a pre-trained SentenceTransformer model as the backbone. The SentenceTransformer remains frozen during training and only the MLP head is finetuned.

TODO - option to load already trained classifier (need to incorperate into prereq's too).

In [ ]:
import numpy as np
from workshop_utilities import set_seed, split_data
set_seed(2022)  # This will ensure reproducibility (at least on CPU!)

In [ ]:
classes

In [ ]:
n_classes = len(classes)
n_train_c = [0] * n_classes
n_test_c = [0] * n_classes

n_train_c[5], n_train_c[11] = 200, 200  # 
n_test_c[5], n_test_c[11] = 100, 100  # 

(x_train, y_train), (x_test, y_test), _ = split_data(dataset.data, dataset.target, n_train_c, n_test_c, seed=0)

In [ ]:
y_train

In [ ]:
emb_train = emb_model(x_train)
plot_clusters(emb_train, y_train, classes, dr_model=umap_model, title='Training data: clustered news topics')

In [ ]:
import torch
from workshop_utilities import Classifier, train_model, eval_model

TRAIN_CLF = False  # Set to TRUE to train classifier, otherwise it will be loaded from disk
filepath = 'classifier'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if TRAIN_CLF:
    # init model
    clf = Classifier().to(device)

    # Train model
    train_model(clf, x_train, y_train, epochs=5, shuffle=True)
    clf.eval()
    
    # Save model
    torch.save(clf.state_dict(), filepath)
else:
    # Load model
    clf = Classifier()
    clf.load_state_dict(torch.load(filepath))
    clf = clf.to(device)
    clf.eval()

_, _ = eval_model(clf, x_train, y_train, shuffle=True)

### Testing the classifier

In [ ]:
emb_test = emb_model(x_test)
plot_clusters(emb_test, y_test, classes, dr_model=umap_model, title='Test data: clustered news topics')

Test some classifier predictions

In [ ]:
idx = 42
print(x_test[idx])

In [ ]:
class_pred = clf([x_test[idx]]).argmax(1)
classes[class_pred]

In [ ]:
idx = 42
print(x_test[idx])

In [ ]:
class_pred = clf([x_test[idx]]).argmax(1)
classes[class_pred]

## 2. Detecting drift

### Introducing drift

Model uncertainty and supervised w/ FET

In [ ]:
n_classes = len(classes)
n_nodrift_c = n_test_c
n_drift_c = [0] * n_classes

n_drift_c[5], n_drift_c[11], n_drift_c[14] = 100, 100, 100  # 

(x_nodrift, y_nodrift), (x_drift, y_drift), _ = split_data(dataset.data, dataset.target, n_nodrift_c, n_drift_c, seed=0)

In [ ]:
emb_drift = emb_model(x_drift)
plot_clusters(emb_drift, y_drift, classes, dr_model=umap_model, title='Drifted data: clustered news topics')

In [ ]:
idx = np.unique(y_drift).astype(int)
np.array(classes)[idx]

### Detecting model drift

If we have labels can monitor model performance directly...

In [ ]:
_, _ = eval_model(clf, x_nodrift, y_nodrift)

In [ ]:
_, _ = eval_model(clf, x_drift, y_drift)

Otherwise we can use model uncertainty as a proxy

In [ ]:
from alibi_detect.cd import ClassifierUncertaintyDrift

def preprocess_batch(x):
    return clf.embedding_model(x)

dd = ClassifierUncertaintyDrift(x_test, clf, 
                                preprocess_batch_fn=preprocess_batch, backend='pytorch', 
                                p_val=.05, preds_type='logits')

In [ ]:
dd.predict(x_nodrift)

In [ ]:
dd.predict(x_drift)['data']

## 3. Detecting drift on the inputs

basic MMDDrift use

Use UMAP to viz drift.

In [ ]:
def preprocess_fn(x):
    x = clf.embedding_model(x)
    return x.cpu().numpy()

In [ ]:
from alibi_detect.cd import MMDDrift
dd = MMDDrift(x_test, backend='pytorch', p_val=.05, preprocess_fn=preprocess_fn)

In [ ]:
dd.predict(x_nodrift)['data']

In [ ]:
dd.predict(x_drift)['data']

## 4. Accounting for context

rgrgrhg

what happens if training 

###  Changing the relative subpopulation prevalence

relative frequency of one or more subpopulations (i.e. news topics) is changing in a way which can be attributed to external events. Importantly, the distribution underlying each subpopulation (e.g. the distribution of *hockey* news itself) remains unchanged, only its frequency changes.

In our example we assume that the World Series and Stanley Cup coincide on the calendar leading to a spike in news articles on respectively baseball and hockey. Furthermore, there is not too much news on Mac or Windows since there are no new releases or products planned anytime soon.

In [ ]:
n_classes = len(classes)
n_nochange_c = 1000 // n_classes  # equally subsample each class from 1000 instances

n_change_c = [50] * n_classes  # 100 of each class (but then mod. below)
n_change_c[4], n_change_c[5] = 25, 25  # few stories on Mac/Windows
n_change_c[9], n_change_c[10] = 75, 75  # more stories on baseball/hockey

(x_nochange, y_nochange), (x_change, y_change), (x_held, y_held) = split_data(dataset.data, dataset.target, n_nochange_c, n_change_c, seed=0)

# Split remaining data into train/ref
idx_ref = np.random.choice(len(x_held), size=1000, replace=False)
x_ref, y_ref = [x_held[_] for _ in idx_ref], y_held[idx_ref]

In [ ]:
emb_ref = emb_model(x_ref)
plot_clusters(emb_ref, y_ref, classes, dr_model=umap_model, title='Reference data: clustered news topics')

### Vanilla MMD detector

In [ ]:
dd = MMDDrift(x_ref, p_val=0.05, preprocess_fn=preprocess_fn)

In [ ]:
dd.predict(x_change)

The MMD detector consistently flags drift (low p-values). Note that this is the expected behaviour since the vanilla MMD detector cannot take any external context into account and correctly detects that the reference and test data do not follow the same underlying distribution.

However... not necesarilly drift we want to detect. Classifier should work fine on this data!


In [ ]:
TRAIN_CLF = True  # Set to TRUE to train classifier, otherwise it will be loaded from disk
filepath = 'classifier_full'
if TRAIN_CLF:
    # init model
    clf_full = Classifier().to(device)

    # Train model
    train_model(clf_full, x_train, y_train, epochs=5, shuffle=True)
    clf_full.eval()
    
    # Save model
    torch.save(clf_full.state_dict(), filepath)
else:
    # Load model
    clf_full = Classifier()
    clf_full.load_state_dict(torch.load(filepath)).to(device)
    clf_full.eval()

_, _ = eval_model(clf_full, x_nochange, y_nochange)
_, _ = eval_model(clf_full, x_change, y_change)

### Context aware MMD detector

We want to detect drift, given...

To achieve this we **condition on the prediction probabilities of the classifier we trained earlier to distinguish each of the 20 different news topics**. We can do this because the prediction probabilities can account for the frequency of occurrence of each of the topics (be it imperfectly given our classifier makes the occasional mistake).

In [ ]:
def context(x):
    """ Condition on classifier prediction probabilities. """
    logits = clf(x)
    softmax_fn = torch.nn.Softmax(dim=-1)
    return softmax_fn(logits).detach().cpu().numpy()

In [ ]:
from alibi_detect.cd import ContextMMDDrift
dd_cad = ContextMMDDrift(x_ref, context(x_ref), p_val=.05, n_permutations=100, 
                         preprocess_fn=preprocess_fn, backend='pytorch')

In [ ]:
dd_cad.predict(x_change, context(x_change))

Hopefully drift isn't detected! (hint: it might be sometimes, as we expect a uniform distribution of p_val)

## Homework!

### Examining detector calibration

Before we set off our experiments, we compute all necessary embeddings and contexts so we don't have to run the embedding model on every loop. 

In [ ]:
n_exp = 5000  # This dataset is going to be split further in the loop, so it needs to be relatively large
idx = np.random.choice(n_all, size=n_exp, replace=False)
x_exp = [dataset.data[_] for _ in idx]

emb_exp, c_exp = emb_model(x_exp).cpu().numpy(), context(x_exp)

In [ ]:
from tqdm import tqdm

n_runs = 50  # number of drift detection runs, each with a different reference and test sample
n_ref, n_test = 1000, 500

p_vals_mmd, p_vals_cad = [], []
for _ in tqdm(range(n_runs)):
    
    # sample data
    idx = np.random.choice(n_exp, size=n_exp, replace=False)
    idx_ref, idx_test = idx[:n_ref], idx[n_ref:n_ref+n_test]
    emb_ref, c_ref = emb_exp[idx_ref], c_exp[idx_ref]
    emb_test, c_test = emb_exp[idx_test], c_exp[idx_test]
    
    # mmd drift detector
    dd_mmd = MMDDrift(emb_ref, p_val=.05, n_permutations=100, backend='pytorch')
    preds_mmd = dd_mmd.predict(emb_test)
    p_vals_mmd.append(preds_mmd['data']['p_val'])
    
    # context-aware mmd drift detector 
    dd_cad = ContextMMDDrift(emb_ref, c_ref, p_val=.05, n_permutations=100, backend='pytorch')
    preds_cad = dd_cad.predict(emb_test, c_test)
    p_vals_cad.append(preds_cad['data']['p_val'])
    
p_vals_mmd = np.array(p_vals_mmd)
p_vals_cad = np.array(p_vals_cad)

The below figure of the [Q-Q (Quantile-Quantile) plots](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot) of a random sample from the uniform distribution *U[0,1]* against the obtained p-values from the vanilla and context-aware MMD detectors illustrate how well both detectors are calibrated. A perfectly calibrated detector should have a Q-Q plot which closely follows the diagonal. Only the middle plot in the grid shows the detector's p-values. The other plots correspond to *n_runs* p-values actually sampled from *U[0,1]* to contextualise how well the central plot follows the diagonal given the limited number of samples.

In [ ]:
from workshop_utilities import plot_qq
plot_qq(p_vals_mmd, 'Q-Q plot MMD detector')
plot_qq(p_vals_cad, 'Q-Q plot Context-Aware MMD detector')

As expected we can see that the context-aware MMD detectors is well-calibrated, but the normal MMD isn't!

The same can be seen using histogram's of p-value:

In [ ]:
from workshop_utilities import plot_hist
p_vals = [p_vals_mmd, p_vals_cad]
title = 'p-value distribution for a change in subpopulation prevalence'
plot_hist(p_vals, title)

Test power can be quantified in a similar way, but here we want to examine... see...

### Changing the subpopulation distribution

See https://docs.seldon.io/projects/alibi-detect/en/latest/examples/cd_context_20newsgroup.html#Changing-the-subpopulation-distribution for an example where the distribution of the subpopulation has actually changed. In this case, we expect the context aware detector is expected to detect drift.

### Changing the context

See 

### Interpretability of detections